In [ ]:
%pip install azure-eventhub

In [ ]:
# This cell generates random temp and humidity data for a defined number of devices
import time
import os
import datetime
import random
import json
from azure.eventhub import EventHubProducerClient,EventData

# Creates an array of devices to send data for
devices = []
for x in range(1, 11):
    devices.append(int(x))
print(devices)

#Get from the Custom App source of your Fabric EventStream (the Event Hub Name and the Connection String-Primary Key)
EventHubName = 'es_9f9c...'
EventHubEndpoint = 'Endpoint=sb://esehblfl...'

# Create a producer client to produce and publish events to the event hub.
producer = EventHubProducerClient.from_connection_string(conn_str=EventHubEndpoint, eventhub_name=EventHubName)

# Update the range to specify the number of iterations, the data provided in the reading JSON packet, and/or the seconds between each batch of data
for y in range(0,6):
    event_data_batch = producer.create_batch() # Create a batch. You will add events to the batch later. 
    for dev in devices:
        # Create a dummy reading.
        reading = {
            'id': dev, 
            'timestamp': str(datetime.datetime.utcnow()),
            'temperature': random.randint(80, 92), 
            'humidity': random.randint(80, 100)
        }
        s = json.dumps(reading) # Convert the reading into a JSON string.
        print(s)
        event_data_batch.add(EventData(s)) # Add event data to the batch.
    producer.send_batch(event_data_batch) # Send the batch of events to the event hub.
    time.sleep(15)
producer.close()

In [ ]:
# This cell sends a filtered subset of rows from a dataframe to an EventStream every x seconds. Use the provided csv url, or create your own dataframe and update the code
import pandas as pd
import time
import os
import datetime
import json
from azure.eventhub import EventHubProducerClient,EventData

#Source data - use this url or create your own mock data/dataframe
url = 'https://raw.githubusercontent.com/hoosierbi/fileshare/main/DeviceTemp/DeviceTemps.csv'

#Get from the Custom App source of your Fabric EventStream (the Event Hub Name and the Connection String-Primary Key)
EventHubName = 'es_847505...'
EventHubEndpoint = 'Endpoint=sb://esehblrhf...'

#Read in the csv, unpivot the data, and filter to a few of the devices
data_raw = pd.read_csv(url)
data_unpivot = pd.melt(data_raw, id_vars='Batch')
data = data_unpivot.rename(columns={'variable': 'Device', 'value': 'Temp'})
data = data[data['Device'].isin(['D003', 'D004', 'D007', 'D011', 'D014', 'D016', 'D017'])]

# Create a producer client to produce and publish events to the event hub.
producer = EventHubProducerClient.from_connection_string(conn_str=EventHubEndpoint, eventhub_name=EventHubName)

# Data has 500 batches of data in it. Iterate through all of them. Update the time.sleep for seconds between batches.
for y in range(1,501):
    event_data_batch = producer.create_batch() # Create a batch. You will add events to the batch later. 
    devices = data[data['Batch'] == y]

    for row in devices.itertuples(index=False):
        reading = {
            'id': row.Device, 
            'timestamp': str(datetime.datetime.utcnow()),
            'temperature': row.Temp
        }
        s = json.dumps(reading) # Convert the reading into a JSON string.
        print(s)
        event_data_batch.add(EventData(s)) # Add event data to the batch.
    producer.send_batch(event_data_batch) # Send the batch of events to the event hub.
    time.sleep(5) #wait time in seconds
producer.close()